In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cPickle, h5py, os

from oc_ica import analysis
reload(analysis)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


<module 'oc_ica.analysis' from '/home/jesse/Development/oc_ica/oc_ica/analysis.pyc'>

# HDF5 Storage

In [4]:
folder = '/home/jesse/Development/results/oc_ica'
a_file = 'a_array-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE.h5'
fit_files = ['comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-2.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-4.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB_F.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM_F.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COHERENCE_redo.h5']
sc_fits = ['comparison_first-8-lambdas_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-SC.h5',
           'comparison_last-9-lambdas_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-SC.h5']

models = [f.split('.')[-2].split('-')[-1] for f in fit_files] + [sc_fits[0].split('.')[-2].split('-')[-1]]
print models

['2', '4', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'COHERENCE_redo', 'SC']


In [5]:
with h5py.File(os.path.join(folder, a_file), 'r') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

In [6]:
colors = ['black', 'black', 'red', 'red', 'green', 'green', 'blue', 'purple']
fmts = ['-', ':', '-', '--', '-', '--', '-', '-']
with h5py.File(os.path.join(folder, fit_files[0]), 'r') as f:
    lambdas = f['lambdas'].value
n_mixtures, n_sources = A_array.shape[2:]
n_iter = A_array.shape[1]

In [7]:
W_fits = np.full((len(A_priors), len(models), lambdas.size, n_iter, n_sources, n_mixtures),
                 np.nan, dtype='float32')
results = np.full((len(A_priors), len(models), lambdas.size, n_iter, 3),
                  np.nan, dtype='float32')
max_angles = np.full((len(A_priors), len(models), lambdas.size, n_iter, n_sources),
                     np.nan, dtype='float32')
other_angles = np.full((len(A_priors), len(models), lambdas.size, n_iter, n_sources**2-n_sources),
                       np.nan, dtype='float32')

In [8]:
for ii, f_name in enumerate(fit_files):
    with h5py.File(os.path.join(folder, f_name), 'r') as f:
        W_fits[:, ii] = np.squeeze(f['W_fits'])
        
loc = 0
for ii, f_name in enumerate(sc_fits):
    with h5py.File(os.path.join(folder, f_name), 'r') as f:
        n_lambdas = f['W_fits'].shape[2]
        W_fits[:, -1, loc:loc+n_lambdas, :10] = np.squeeze(f['W_fits'])
        loc += n_lambdas

In [9]:
reload(analysis)
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                try:
                    A = A_array[ii, ll]
                    W = W_fits[ii, jj, kk, ll]
                    assert (not np.isnan(A.sum())) and (not np.isnan(W.sum()))
                    hd, (pd, ma, oa, mod) = analysis.recovery_statistics_AW(A, W)
                    results[ii, jj, kk, ll] = [hd, pd, mod]
                    max_angles[ii, jj, kk, ll] = ma
                    other_angles[ii, jj, kk, ll] = oa
                except (ValueError, AssertionError):
                    print ii, jj, kk, ll

0 0 14 16
0 0 14 27
0 0 14 29
0 0 14 33
0 0 14 38
0 0 15 1
0 0 15 2
0 0 15 6
0 0 15 8
0 0 15 9
0 0 15 10
0 0 15 11
0 0 15 16
0 0 15 17
0 0 15 18
0 0 15 19
0 0 15 21
0 0 15 25
0 0 15 26
0 0 15 27
0 0 15 29
0 0 15 31
0 0 15 34
0 0 15 35
0 0 15 36
0 0 15 37
0 0 15 38
0 0 15 39
0 0 16 0
0 0 16 1
0 0 16 2
0 0 16 3
0 0 16 4
0 0 16 6
0 0 16 7
0 0 16 8
0 0 16 9
0 0 16 10
0 0 16 11
0 0 16 12
0 0 16 13
0 0 16 14
0 0 16 15
0 0 16 16
0 0 16 17
0 0 16 18
0 0 16 19
0 0 16 21
0 0 16 22
0 0 16 23
0 0 16 24
0 0 16 25
0 0 16 26
0 0 16 27
0 0 16 28
0 0 16 29
0 0 16 30
0 0 16 31
0 0 16 32
0 0 16 33
0 0 16 35
0 0 16 36
0 0 16 37
0 0 16 38
0 0 16 39
0 1 15 4
0 1 15 22
0 1 16 0
0 1 16 1
0 1 16 2
0 1 16 3
0 1 16 7
0 1 16 8
0 1 16 9
0 1 16 10
0 1 16 12
0 1 16 13
0 1 16 14
0 1 16 15
0 1 16 16
0 1 16 18
0 1 16 19
0 1 16 20
0 1 16 21
0 1 16 22
0 1 16 23
0 1 16 24
0 1 16 25
0 1 16 26
0 1 16 27
0 1 16 28
0 1 16 29
0 1 16 30
0 1 16 31
0 1 16 32
0 1 16 33
0 1 16 35
0 1 16 36
0 1 16 38
0 1 16 39
0 7 0 10
0 7 0 11
0 7 

In [14]:
n_metrics = 2
f, axes = plt.subplots(len(A_priors), n_metrics, figsize=(4*n_metrics, 3*len(A_priors)))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    #(ad, ae, ap, aa, am) = a
    (ae, am) = a
    for jj, m in enumerate(models):
        if ii == 0:
            label = str(m)
            if label != 'SC':
                label = label.title()
            label = label.replace('_Redo', '')
        else:
            label = None
        ma = np.nanmedian(max_angles[ii, jj, :, :], axis=-1)
        oa = np.nanmedian(other_angles[ii, jj, :, :], axis=-1)
        da = (ma - oa).mean(axis=1)
        dist = np.nanmean(results[ii, jj, :, :, 0], axis=1)
        delta = np.nanmean(results[ii, jj, :, :, 1], axis=1)
        distp = np.nanmean(results[ii, jj, :, :, 2], axis=1)
        #ad.semilogx(lambdas, dist, fmts[jj], label=label, c=colors[jj])
        #ad.semilogx(lambdas, dist, '.', c=colors[jj])
        ae.semilogx(lambdas, delta, fmts[jj], label=label, c=colors[jj])
        ae.semilogx(lambdas, delta, '.', c=colors[jj])
        ae.minorticks_off()
        #ap.semilogx(lambdas, distp, fmts[jj], label=label, c=colors[jj])
        #ap.semilogx(lambdas, distp, '.', c=colors[jj])
        #aa.semilogx(lambdas, da, fmts[jj], label=label, c=colors[jj])
        #aa.semilogx(lambdas, da, '.', c=colors[jj])
        am.semilogx(lambdas, np.nanmean(ma, axis=1), fmts[jj], label=label, c=colors[jj])
        am.semilogx(lambdas, np.nanmean(ma, axis=1), '.', c=colors[jj])
        am.minorticks_off()
    #ad.legend(loc='lower left', bbox_to_anchor=(.2, .6, 1.1, 1.1),
    #          ncol=2, prop={'size': 8})
    ae.legend(loc='upper right', ncol=2, prop={'size': 8})

    #ad.set_title(p)
    #ad.set_xlabel('Lambda')
    #ad.set_ylabel('Hellinger Distance')
    ae.set_title(p)
    ae.set_ylabel(r'$\Delta P$')
    ae.set_ylim([0, 200])
    #ap.set_title(p)
    #ap.set_xlabel('Lambda')
    #ap.set_ylabel('Hellinger Dist. Max vs. Other')
    #aa.set_title(p)
    #aa.set_xlabel('Lambda')
    #aa.set_ylabel('Diff. in Median Max vs. Other')
    am.set_title(p)
    am.set_ylabel(r'Median of $p_{\mathrm{min}}$')
    am.set_ylim([0, 80])
    
ae.set_xlabel(r'$\lambda$')
am.set_xlabel(r'$\lambda$')


f.tight_layout()
plt.savefig('model_fit_comparison.pdf')
plt.show()
plt.close()

In [11]:
f, axes = plt.subplots(len(A_priors), len(models), figsize=(6*(len(models)), 2*len(A_priors)))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    for jj, (m, ax) in enumerate(zip(models, a)):
        if (ii == 3) and (jj == 1):
            labels = ['Max dist.', 'Max median', 'Max mode', 'Other dist.', 'Other median', 'Other mode']
        else:
            labels = 6 * [None]
        ma = max_angles[ii, jj, 4, :]
        ma = ma[np.logical_not(np.isnan(ma))]
        oa = other_angles[ii, jj, 4, :]
        oa = oa[np.logical_not(np.isnan(oa))]
        m_median = np.nanmedian(ma)
        o_median = np.nanmedian(oa)
        md, bins = analysis.angle_histogram(ma)
        od, bins = analysis.angle_histogram(oa)
        m_mode = md.argmax()
        o_mode = od.argmax()
        ax.semilogy(bins[:-1], md, drawstyle='steps', color='red', lw=1.5, label=labels[0])
        ax.axvline(m_median, color='red', ls='dotted', label=labels[1])
        ax.axvline(m_mode, color='red', ls='dashed', label=labels[2])
        ax.semilogy(bins[:-1], od, drawstyle='steps', color='black', lw=1.5, label=labels[3])
        ax.axvline(o_median, color='black', ls='dotted', label=labels[4])
        ax.axvline(o_mode, color='black', ls='dashed', label=labels[5])
        if (ii == 3) and (jj == 1):
            ax.legend(loc='center', prop={'size': 8})

        ax.set_title(str(m))
        if ii == len(A_priors)-1:
            ax.set_xlabel('Angle')
        if jj == 0:
            ax.set_ylabel(p)
        ax.set_xticks([0, 45, 90])
        ax.set_yticks([1e-6, 1e-3, 1e0])

f.tight_layout()
plt.savefig('permutation_comparison.pdf')
plt.show()
plt.close()

# Pickle Storage

In [3]:
fname = ''
with open(fname, 'r') as f:
    data = cPickle.load(f)

In [22]:
A_dict, W_dict, W_fits, min_ks, results = data
n_mixtures, n_sources = A_dict[A_dict.keys()[0]][0].shape
n_iter = W_fits.shape[3]
results = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, 3))
max_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources))
other_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources**2-n_sources))

In [23]:
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        A = A_dict[p][jj]
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                W = W_fits[ii, jj, kk, ll]
                hd, (pd, ma, oa, mod) = analysis.recovery_statistics_AW(A, W)
                results[ii, jj, kk, ll] = [hd, pd, mod]
                max_angles[ii, jj, kk, ll] = ma
                other_angles[ii, jj, kk, ll] = oa